# DERİN ÖĞRENME İLE MASKE TANIMA

# Veri Seti
2863 maskeli,
1743 maskesiz

Veri seti adresi: https://github.com/cabani/MaskedFace-Net 

# 1) Gerekli Kütüphanelerin İmport İşlemleri ve Yüklenmesi

In [ ]:
import torch 
import os
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import cv2
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data.dataset import Dataset
from torchvision.transforms import Compose, Resize, ToPILImage, ToTensor

from torch.utils.data.dataloader import DataLoader
from torchvision.utils import make_grid

# 2) Veri Setinin Yüklenmesi

In [ ]:
classes = ['Non-Masked', 'Masked']

REBUILD_DATA = True

# Görüntülerin olduğu path bilgisinin tutulması
if REBUILD_DATA: 
    data_path = Path('/content/drive/MyDrive/Dataset/self-built-masked-face-recognition-dataset')
    maskPath = data_path/'AFDB_masked_face_dataset'
    nonMaskPath = data_path/'AFDB_face_dataset'
    maskDF = pd.DataFrame() 
    path_dirs = [ [maskPath,1],[nonMaskPath,0] ] #path and label
    if not os.path.exists(data_path):
        raise Exception("The data path doesn't exist")

# 2.1) Image Size ve Label Etiket Değerlerinin Belirlenmesi

In [ ]:
class MaskvNoMask():
    IMG_SIZE = 100
    LABELS = {'NON_MASKED': 0, 'MASKED': 1}
    training_data = []
    
    count = 0
    
    def make_training_data(self):
        """
    Görüntülerin dosya konumunu açar ve ilgili görüntülerin label değerlerini belirler.
    
    Parametreler:
    
    self: https://micropyramid.com/blog/understand-self-and-__init__-method-in-python-class/
        """
        for data_dir, label in path_dirs:
            print('Reading from: ',label)
            for folder in tqdm(os.listdir(data_dir)):
                folder_path = os.path.join(data_dir, folder)
                for imgpath in os.listdir(folder_path):
                    self.count += 1                       
                    img_path = os.path.join(folder_path, imgpath)
                    try:
                        img = cv2.imread(img_path) #imread fonksiyonu yardımı ile görüntülerin okunması
                        img = cv2.resize(img, (self.IMG_SIZE,self.IMG_SIZE)) # görüntülerin yeniden boyutlandırılması
                        self.training_data.append([np.array(img), label])
                        self.count +=1 # eklenen görüntülerin saydırılması
                        
                        # Maskeli ve maskesiz görüntülerin sayılması ?
                        if label == 1:
                            self.LABELS['MASKED'] += 1
                        if label == 0:
                            self.LABELS['NON_MASKED'] +=1
                    
                    except:
#                         raise Exception('error: {}'.format(img_path))
                        pass
            print(self.LABELS)
#                         raise Exception('error occured while reading , {}'.format(os.path.join(maskPath, os.path.join(subject, imgPath))))
                        
        #veri setinde rastgelelik oluşturulması işlemi
        np.random.shuffle(self.training_data)

# Veri setinin fonksiyona verilmesi        
if REBUILD_DATA:
    maskvnomask = MaskvNoMask()
    maskvnomask.make_training_data()
    training_data = maskvnomask.training_data

In [ ]:
# Görüntülere ait boyut bilgisi
training_data[0][0].shape #training_data.shape, 'list' object has no attribute 'shape'

In [ ]:
# Görüntüye ait matris değerlerinin getirilmesi
training_data[0:1]

# 3) Veri Setinden Örnekler Gösterilmesi

In [ ]:
#Görüntünün imshow yardımı ile görselleştirilmesi
plt.imshow(training_data[22][0])

In [ ]:
# Görüntüye ait label değeri
print(classes[training_data[1][1]])

# 4) Veri Kümesinin Oluşturulması ve Görselleştirilmesi

In [ ]:
class MaskDataset(Dataset):
        """ Masked faces dataset
        0 = 'no mask'
        1 = 'mask'
        """
        def __init__(self, train_data):
            """
        Görselleri tensor formatına çevirir.
    
        Parametreler:
    
        self: https://micropyramid.com/blog/understand-self-and-__init__-method-in-python-class/
        
        train_data : eğitim veri seti
            """
            self.train_data = train_data
            #https://pytorch.org/vision/stable/transforms.html
            self.transformations = Compose([
                ToTensor(),
            ])
        
        def __getitem__(self, key): #for understanding get item = https://www.programmersought.com/article/98542425111/
            """
        Tensor formatına çevrilen görüntülerin index bilgisini getirir.
    
        Parametreler:
    
        self: https://micropyramid.com/blog/understand-self-and-__init__-method-in-python-class/
        
        key: train_data'da bulunan her bir görsel
    
        Return:
    
        self.transformations(self.train_data[key][0]): Görüntü
    
        torch.tensor(self.train_data[key][1]: Label bilgisi
            """
            if isinstance(key, slice):
                raise NotImplementedError('slicing is not supported')                    
            return [
                self.transformations(self.train_data[key][0]),
                torch.tensor(self.train_data[key][1]) 
            ]
        # magic method : https://stackoverflow.com/questions/2481421/difference-between-len-and-len
        def __len__(self):
            return len(self.train_data)

In [ ]:
myDataset = MaskDataset(training_data)

In [ ]:
def show_example(data):
    """
    Tensor formatına çevrilen görüntülerin görselleştirilmesi.
    
    Parametreler:
    
    data : Görüntünün index bilgisi
    
    """
    img, label = data
    print('Label: ', classes[int(label.item())], "("+str(label.item())+")")
    plt.imshow(img.permute(1, 2, 0))

In [ ]:
myDataset[36]

In [ ]:
show_example(myDataset[36])

In [ ]:
img, label = myDataset[36]
print(img.shape)
print(label)

# 4.1) Train - Validation Split İşleminin Gerçekleştirilmesi

In [ ]:
val_size = 1000
train_size = len(myDataset) - val_size

train_ds, val_ds = torch.utils.data.random_split(myDataset, [train_size, val_size])#train_test_split
len(train_ds), len(val_ds)

In [ ]:
show_example(val_ds[100])

In [ ]:
#Data loader açıklaması 
#Veri setinin mini gruplara ayrılması

batch_size = 32

train_dl = DataLoader(train_ds, batch_size=batch_size*2, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=batch_size*2)

In [ ]:
def show_batch(dl):
    """
    Veri setinden örnek görüntüleri görselleştirir.
    
    Parametreler:
    
    dl: Görüntü
    """
    
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
        break

In [ ]:
show_batch(train_dl)

# 5) Optimizasyon ve Değerlendirme Metriklerinin Ayarlanması

In [ ]:
def accuracy(outputs, labels):
    """
    Accuracy değerini hesaplar
    
    Parametreler:
    
    outputs: Tahminlenen görüntüler
    labels: Etiket değerleri
    
    Return:
    
    torch.tensor(torch.sum(preds == labels).item() / len(preds)) : (Tahmin edilen label değerleri = görüntüye ait label değerleri) / Tüm tahminler' in toplamı
    """
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        """
        Modele ait loss değerini hesaplar.
        
        Parametreler:
        
        self: https://micropyramid.com/blog/understand-self-and-__init__-method-in-python-class/
        
        batch: küçük veri grubu
        
        Return:
        
        loss: hata değerlendirme metriği
        """
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels.long()) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        """
        Modele ait val_loss ve val_acc değerini hesaplar.
        
        Parametreler:
        
        self: https://micropyramid.com/blog/understand-self-and-__init__-method-in-python-class/
        
        batch: küçük veri grubu
        
        Return:
        
        val_loss: 
        
        val_acc :
        
        loss.detach() -> https://stackoverflow.com/questions/56816241/difference-between-detach-and-with-torch-nograd-in-pytorch
        
        """
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels.long())   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        """
        Küçük veri gruplarının val_loss ve val_acc değerlerinin ortalamasını hesaplar.
        
        Parametreler:
        
        self: https://micropyramid.com/blog/understand-self-and-__init__-method-in-python-class/
        
        outputs: Tahminlenen görüntüler
        
        Return:
        
        val_loss: 
        
        val_acc :
        
        item() -> https://www.programiz.com/python-programming/methods/dictionary/items
        """
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        """
        Hesaplanan train_loss,val_loss ve val_acc değerlerini bastırır.
        
        Parametreler:
        
        self: https://micropyramid.com/blog/understand-self-and-__init__-method-in-python-class/
        
        epoch: Epoch sayısı, model eğitilirken verilerin modelden kaç kez geçiş yapacağını belirtir.
        
        result: Sonuç
        """
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
    def predict():
        """
        Tahmin değerlerinden iki sınıf arasında max olan değeri alır.
        """
        pred = model(img)
        _, preds = torch.max(pred, dim=1)

# 6) Modelleme

In [ ]:
class MaskDetection(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 100, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(100, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 128 x 8 x 8

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 256 x 4 x 4

            nn.Flatten(), 
            nn.Linear(160000, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 2))
        
    def forward(self, xb):
        """
        forward method : https://discuss.pytorch.org/t/forward-method-call-for-weight-training/84314
        """
        return self.network(xb) # xb = weights and biases

In [ ]:
class MaskDetection1(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 64 x 8 x 8

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 128 x 4 x 4

            nn.Flatten(), 
            nn.Linear(80000, 1024),
            nn.ReLU(),
            nn.Linear(1024, 2))
        
            
        
    def forward(self, xb):
        return self.network(xb) # xb = weights and biases

# 6.2) Ekran Kartı Ayarlarının Yapılması

In [ ]:
# Eğitim esnasında ekran kartının aktif olup olmadığının kontrol edilmesi 
torch.cuda.is_available()

In [ ]:
def get_default_device():
    """
    Ekran kartı kontrollerini gerçekleştirir.
    """
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [ ]:
# Ekran kartı bilgisi
device = get_default_device()
device

In [ ]:
def to_device(data, device):
    """
    Görüntüye ait tensor değerlerini ekran kartına aktarır.
    
    Parametreler:
    
    data: Tensor formatına çevrilmiş görüntüler
    
    device: Ekran kartı
    
    Return:
    
    device: Ekran kartı
    non_blocking -> https://jovian.ai/forum/t/purpose-of-non-blocking-true-in-tensor-to/14760
    """
    #move tensors to device
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [ ]:
# Görüntünün boyut bilgisini bastırır.

for images, labels in train_dl:
    print(images.shape)
    images = to_device(images, device)
    print(images.device)
    break

# 6.3) Veri Setinin Ekran Kartına Aktarılması

In [ ]:
class DeviceDataLoader():
    #move data to a device
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        #Yield a batch of data after moving it to device
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        #Number of batchs
        return len(self.dl)

In [ ]:
# Train ve Validation setlerinin aktarılması
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)

In [ ]:
# xb = Ekran kartının çalışma bilgisi
# yb = mini veri grubuna (batch_size) ait görüntülerin label değerleri
for xb, yb in val_dl:
    print('xb.device:', xb.device)
    print('yb:', yb)
    break

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    """
    https://discuss.pytorch.org/t/model-train-and-model-eval-vs-model-and-model-eval/5744
    """
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    """
    
    Modeli eğitir
    
    
    Parametreler : 
    
    epochs: Epoch sayısı, model eğitilirken verilerin modelden kaç kez geçiş yapacağını belirtir.
    
    lr: Öğrenme oranı
    
    model.parameters: Moel parametreleri
    
    train_loader: Eğitim veri seti 
    
    val_loader: Validasyon veri seti 
    
    opt_func: Optimizasyon Fonksiyonu
    
    
    Return:
    
    history: Modele ağit eğitim verileri
    """
    history = []
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    for epoch in range(epochs): 
        print('epoch: ', epoch)
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
# Model (on GPU)
model = MaskDetection1()
to_device(model, device)

In [ ]:
# Model ağırlıklarının kaydedilmesi
checkpoint_path = os.path.join(os.getcwd(), "checkpoints")
torch.save(model.state_dict(), checkpoint_path)

In [ ]:
history = [evaluate(model, val_dl)]
history

In [ ]:
epoch=4
history = fit(epoch, 1e-3, model, train_dl, val_dl)

# 6.4) Sonuçların Görselleştirilmesi

In [ ]:
def plot_losses(history):
    """
    Modele ait Validation Loss grafiğini çizdirir.
    
    Parametreler:
    
    history: Modele ağit eğitim verileri
    """
    losses = [x['val_loss'] for x in history]
    plt.plot(losses, '-x')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title('Validation Loss');

In [ ]:
def plot_accuracies(history):
    """
    Modele ait Validation Accuracy grafiğini çizdirir.
    
    Parametreler:
    
    history: Modele ağit eğitim verileri
    """
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Validation Accuracy');

In [ ]:
plot_losses(history)

In [ ]:
plot_accuracies(history)

# 7) Model Başarı Değerlendirme

# 7.1) Confusion Matrix

In [ ]:
nb_classes = 2

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(val_dl):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

# 8) Modelin Test Edilmesi

In [ ]:
import matplotlib.image as mpimg
def singleImage(path, label= None, show= False):
    img = cv2.imread(path)
    assert img is not None,"Immage wasn't read properly"
    img = cv2.resize(img, (100, 100))
    img = torch.from_numpy(img)
    img = img.permute((2, 0,1)) # model expects image to be of shape [3, 100, 100]
    img = img.unsqueeze(dim=0).float() # convert single image to batch [1, 3, 100, 100]
    img = img.to('cuda') # Using the same device as the model
    pred = model(img)
    _, preds = torch.max(pred, dim=1)
    print(classes[preds.item()])
    #plt.imshow(img.squeeze(dim=0).permute((1,2,0)).to('cpu'))

    if show:
        plt.imshow(mpimg.imread(path))
        print("the image is :" + classes[preds.item()])

In [ ]:
singleImage('/content/drive/MyDrive/Dataset/self-built-masked-face-recognition-dataset/AFDB_face_dataset/hedujuan/1_0_hedujuan_0134.jpg', show=True)